<a href="https://colab.research.google.com/github/TM0428/wsd2020/blob/main/system_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 必要パッケージのインストール

In [1]:
!pip install -U beautifulsoup4

     |████████████████████████████████| 122kB 22.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [3]:
!pip install gensim

In [4]:
# get html
import requests
import time
from pathlib import Path

def dirsName(name: str):
    return f"data/{name}/"

def fileName(name: str, id: str, extension: str="html")->str:
    return dirsName(name) + id.replace('/','_') + "." + extension #idの階層がローカルの階層にならないように

def urlName(article_url: str, article_id: str)->str:
    return article_url + article_id

def main():
    contents = [ # name, domain, article_urlは手動でせざるを得ないか、article_idを自動で取ってこられるようにしたい
        {
            "name": "sejuku",
            "domain":"https://www.sejuku.net/",
            "article_url":"https://www.sejuku.net/blog/",
            "article_id": ['33294', '3766', '105909'] # 記事idは https://www.sejuku.net/blog/curriculums-python から適当にとってきた。実際は https://www.sejuku.net/blog/archive/page/* から適当に取ってくるといいかもしれない
        },
        {
            "name": "techacademy",
            "domain":"https://techacademy.jp/",
            "article_url":"https://techacademy.jp/magazine/",
            "article_id": ['15508', '15636', '15571'] # 記事idは https://techacademy.jp/magazine/category/programming/python/page/44 
        },
        {
            "name": "note_nkmk",
            "domain":"https://note.nkmk.me/",
            "article_url":"https://note.nkmk.me/",
            "article_id": ['python-union-find', 'python-numpy-sin-con-tan', 'python-list-2d-sort'] # 記事idは https://note.nkmk.me/ の新着記事・更新記事から適当にとってきた
        },
        # https://docs.python.org/ はスクレイピングだめらしい
        
        #{
        #    "name": "python_docs",
        #    "domain":"https://docs.python.org/",
        #    "article_url":"https://docs.python.org/ja/3/library",
        #    "article_id": ['pathlib', 'random', 're'] # 記事idは https://docs.python.org/ja/3/library/ から適当にとってきた
        #},
        {
            "name": "qiita",
            "domain": "https://qiita.com/",
            "article_url": "https://qiita.com/", # https://qiita.com/をarticle_urlにするかhttps://qiita.com/user/をarticle_urlにするかは意見が分かれそう
            "article_id": ["kuroitu/items/f18acf87269f4267e8c1", "paleo_engineer/items/fdbde62df4394ddcaf11", # 記事idは https://qiita.com/kai_kou/items/180a91fd88dbbbd746f6 から適当に取ってきた
                            "UTOG/items/c77c75a6aa61ee93fc6d"] # idの階層がファイル名の階層になると階層構造が複雑になるので変換する必要がある
        }
        
    ]
    res = {}
    for content in contents:
        print(content)
        res[content['domain']] = []
        for id in content["article_id"]:
            filename = fileName(content["name"], id) # ローカル保存用のファイル名
            print(filename)
            file_path = Path(filename)
            print(file_path.exists())
            html_code:str
            if not file_path.exists(): # 1回getしたらローカルに保存して、複数回getしない
                urlname = urlName(content['article_url'], id) # get用のurl名
                print(urlname)
                html = requests.get(urlname) # getしたhtml(bytes)
                html_code = html.content.decode('utf-8') #getしたhtml(string)
                dir_path = Path(dirsName(content['name']))
                if not dir_path.exists():
                    dir_path.mkdir(mode=0o777,parents=True,exist_ok=False) # ディレクトリを作成
                with open(filename, mode='w', encoding='utf-8') as f:
                    f.write(html_code) # ローカルに保存
                time.sleep(3) # 適度なインターバル
            with open(filename, mode='r', encoding='utf-8') as f:
                html_code = f.read()
                res[content['domain']].append(html_code)
            
    return res




if __name__=="__main__":
    a = main()
    print(len(a['https://www.sejuku.net/'][0]))
    print(len(a['https://techacademy.jp/'][1]))
    print(len(a['https://note.nkmk.me/'][2]))
    print(len(a['https://qiita.com/'][0]))
    


{'name': 'sejuku', 'domain': 'https://www.sejuku.net/', 'article_url': 'https://www.sejuku.net/blog/', 'article_id': ['33294', '3766', '105909']}
data/sejuku/33294.html
False
https://www.sejuku.net/blog/33294
data/sejuku/3766.html
False
https://www.sejuku.net/blog/3766
data/sejuku/105909.html
False
https://www.sejuku.net/blog/105909
{'name': 'techacademy', 'domain': 'https://techacademy.jp/', 'article_url': 'https://techacademy.jp/magazine/', 'article_id': ['15508', '15636', '15571']}
data/techacademy/15508.html
False
https://techacademy.jp/magazine/15508
data/techacademy/15636.html
False
https://techacademy.jp/magazine/15636
data/techacademy/15571.html
False
https://techacademy.jp/magazine/15571
{'name': 'note_nkmk', 'domain': 'https://note.nkmk.me/', 'article_url': 'https://note.nkmk.me/', 'article_id': ['python-union-find', 'python-numpy-sin-con-tan', 'python-list-2d-sort']}
data/note_nkmk/python-union-find.html
False
https://note.nkmk.me/python-union-find
data/note_nkmk/python-nump

In [5]:
# html to text
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import re

# text_setを生成する
# text_setは各葉ノードのテキストと木構造を表す
def soup2textset(tag, text_set, tag_string):
    for content in tag.contents:
        if isinstance(content, NavigableString):
            #content.string = content.string.replace('\xa9', '(c)').replace('\xa0', '')
            text_set.add(tag_string + content.string)
        elif str.lower(content.name) in ["script", "meta", "style", "header", "footer"]:
            pass
        else:
            #print(content.name)
            soup2textset(content, text_set, tag_string + str.lower(content.name))
    return text_set

# text_setに一致する葉ノードを削除する
def removetags(tag, remove_text_set, tag_string):
    remove_lists = []
    for content in tag.contents:
        if isinstance(content, NavigableString):
            if (tag_string+content.string) in remove_text_set:
                remove_lists.append(content)
        elif str.lower(content.name) in ["script", "meta", "style", "header", "footer"]:
            remove_lists.append(content)
        else:
            removetags(content, remove_text_set, tag_string + str.lower(content.name))
    for content in remove_lists:
        content.extract()

from typing import List
# Webページを取得して解析する
# def html2text(htmls: list(str)) -> list(str):
# 同じドメインのhtml(string)群を受け取って、タグを削除したstringのlistを返す
def html2text(htmls: List[str]) -> List[str]:
    # 文字列抽出
    soups = [BeautifulSoup(html.replace("</li>", "\n</li>"), "html.parser") for html in htmls]
    #bodies = [soup.find("body").text for soup in soups]

    bodies = [soup.find("body") for soup in soups]

    # 重複の削除
    if 2 <= len(bodies):
        # 重複要素の列挙
        set0 = soup2textset(bodies[0], set(), "")
        set1 = soup2textset(bodies[1], set(), "")
        remove_set = frozenset(set0 & set1)
        if 3 <= len(bodies):
            set2 = soup2textset(bodies[2], set(), "")
            remove_set = frozenset(remove_set | (set2 & set0)  | (set2 & set1))
        #print(remove_set)
        # 重複要素の削除
        for i in range(len(bodies)):
            removetags(bodies[i], remove_set, "")

    #soup => textの変換
    for i in range(len(bodies)):
        bodies[i] = bodies[i].text

    # 空行削除
    emptyline_regex = re.compile('\n\\s*\n')
    for i in range(len(bodies)):
        bodies[i] = emptyline_regex.sub('\n', bodies[i])

    return bodies

def test():
    import os
    html_names = os.listdir("./data/sejuku")
    htmls = [open("./data/sejuku/" + html, 'r', encoding="utf-8").read() for html in html_names]
    bodies = html2text(htmls)

    print(html_names[0])
    print(bodies[0][:100])
    print("........")
    print(bodies[0][-100:])
    print("====================")
    print(html_names[1])
    print(bodies[1][:100])
    print("........")
    print(bodies[1][-100:])
    print("====================")
    print(html_names[2])
    print(bodies[2][:100])
    print("........")
    print(bodies[2][-100:])
    print("====================")
    # print(bodies[0].replace('\xa9', '(c)').replace('\xa0', ''))

if __name__=="__main__":
    test()



105909.html

Pythonの動かし方がいまいちわからない……
実行したらエラーが発生したけど、どうすればいいの？
おすすめの実行環境があったら教えてほしいこれからPythonを学習しようとしている初心者の方にとっ
........
リーランスエンジニアとして仕事獲得するまでの全手順
現役生徒500名以上が在籍する弊社のプログラミング学習プランをあなたに合わせて最短1分で診断します。プログラミング学習プランの診断を受ける（無料） 
33294.html

Pythonを自分のPCにインストールしたいけどどうやったらいいの？
Pythonのバージョンを確認するには？と疑問を持たれている方もいるのではないでしょうか？言語を本格的に学ぶためには環境を構築す
........
だし暫く何も足さない）がいいんじゃないかというメモ - min.t (ミント)未経験からフリーランスエンジニアとして仕事獲得するまでの全手順Pythonとは？特徴やメリット・勉強法を解説【初心者向け】
3766.html
Cloud9を導入すれば、簡単にPythonの開発環境を構築できます！Python学習したい！そう思ってもプログラミングを始めるところにすらたどり着けないという方もいらっしゃるかと思います。今回はCl
........
ebデータベースハーバードで世界の教育を学んで分かった「好奇心」を伸ばす2つの秘訣プログラミングを始める前に知っておきたい基礎知識と3つの鉄則プログラミングを独学で学習する方法と挫折をなくす3つのコツ


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import MeCab
from gensim.models.doc2vec import Doc2Vec
import os


class Embedder:
    def __init__(self, model_path="/content/drive/Shareddrives/システム設計構築演習/jawiki.doc2vec.dbow300d/jawiki.doc2vec.dbow300d.model"):
        """
        constructor

        Args:
            model_path (str, optional): Defaults to "jawiki.doc2vec.dbow300d.model".
        """
        self.model = Doc2Vec.load(model_path)

    def _tokenize(self, text):
        """
        分かち書きを行うメソッド

        Args:
            text (str): 入力文字列

        Returns:
            list[str]: 分かち書き済み単語リスト
        """
        wakati = MeCab.Tagger("-Owakati")
        wakati.parse("")
        return wakati.parse(text).strip().split()

    def doc2vec(self, text):
        """
        doc2vec

        Args:
            text (str): string

        Returns:
            numpy.array: embedded vector
        """
        return self.model.infer_vector(self._tokenize(text))


def main():
    text = 'はじめに今まで自分が使ってきた中で、これは生産性が爆上げする！と思うものを厳選しました是非最後までご覧ください'
    embedder = Embedder()
    print(embedder.doc2vec(text))


if __name__ == "__main__":
  # データ一覧
  html_names = os.listdir("./data/sejuku")
  htmls = [open("./data/sejuku/" + html, 'r', encoding="utf-8").read() for html in html_names]
  bodies = html2text(htmls)
  embedder = Embedder()
  for body in bodies:
    print(embedder.doc2vec(body))

[-4.50162590e-02  1.20037198e-01  3.23985994e-01  1.84358746e-01
 -1.73363835e-01  4.65460926e-01 -1.71287328e-01  1.75518170e-01
 -1.13935024e-01 -1.84094831e-01  3.75749946e-01 -2.01617435e-01
 -3.95487398e-01 -5.04127085e-01  3.57835263e-01  1.47552490e-01
  1.71659365e-01 -1.53158158e-01 -3.82211626e-01 -8.12116683e-01
 -5.59970737e-01  6.89779734e-03  2.92615324e-01  5.86417854e-01
 -1.14900696e+00 -1.89431876e-01 -2.77236134e-01  2.09360108e-01
 -1.03527093e+00  1.09677768e+00 -3.46465319e-01 -6.94154680e-01
 -2.18538895e-01  2.84881771e-01  9.03616309e-01 -5.27385712e-01
 -4.49349254e-01  6.99098647e-01  4.47844982e-01 -5.13688087e-01
 -8.04726422e-01  1.00735202e-01 -1.97084337e-01  7.57810920e-02
  2.99479574e-01  3.97111952e-01  1.37740767e+00 -7.81304181e-01
  7.11947918e-01  6.21662259e-01 -1.19138304e-02 -5.12070715e-01
 -1.59365028e-01 -9.13798928e-01  2.54448950e-01 -3.84475857e-01
  5.54300725e-01  4.46786806e-02 -8.13451827e-01  5.80432236e-01
  7.93553948e-01 -2.71598